In [1]:
from oracles import Oracle
from FISTA import FISTA
from APG import AcceleratedProximalGD
import pandas as pd
import numpy as np
from xml.etree import ElementTree as ET

In [2]:
df_start = pd.read_table('data/data_2014-03-31.dat')
df_end = pd.read_table('data/data_2014-06-30.dat')
df = pd.merge(df_start, df_end, how='inner', on='Date')

In [3]:
n_stocks = len(df)

In [4]:
n_stocks

114

In [5]:
tree = ET.parse('data/data_2014-03-31.xml')
root = tree.getroot()

weights = {}
for item in root.findall('./anchors/anchor/anchor_point'):
    weights[item.get('stock')] = float(item.get('value'))

starting_weights = np.zeros(len(df))
i = 0
for stock_name in df.Date:
    if stock_name in weights.keys():  
        starting_weights[i] = weights[stock_name]
    else:
        starting_weights[i] = 0
    i += 1

In [6]:
tree = ET.parse('data/data_2014-06-30.xml')
root = tree.getroot()

weights = {}
for item in root.findall('./anchors/anchor/anchor_point'):
    weights[item.get('stock')] = float(item.get('value'))

final_weights = np.zeros(len(df))
i = 0
for stock_name in df.Date:
    if stock_name in weights.keys():  
        final_weights[i] = weights[stock_name]
    else:
        final_weights[i] = 0
    i += 1
weights = final_weights

In [7]:
np.sum(starting_weights)

0.93871238946945601

In [8]:
np.sum(final_weights)

0.90288577317514396

In [9]:
starting_weights = starting_weights[1:] / np.sum(starting_weights)
final_weights = final_weights[1:] / np.sum(final_weights)

In [10]:
incomes = np.array(df.values[1:, 1:], dtype=np.float32)

In [11]:
mean_income = np.mean(incomes, axis=1)
covariance_income = np.cov(incomes)

In [12]:
oracle = Oracle(Sigma=covariance_income, income=mean_income, lambd=0.1, rho=1)

In [14]:
AcceleratedProximalGD(oracle, x_0=np.random.uniform(size=len(mean_income)))

SolverError: Solver 'ECOS' failed. Try another solver.